Download the enron dataset

In [ ]:
print("Downloading the Enron dataset (this may take a while)")
print("To check on progress, you can cd up one level, then execute <ls -lthr>")
print("Enron dataset should be last item on the list, along with its current size")
print("Download will complete at about 1.82 GB")

import requests
url = "https://www.cs.cmu.edu/~./enron/enron_mail_20150507.tar.gz"
filename = "../enron_mail_20150507.tar.gz"
with open(filename, "wb") as f:
    r = requests.get(url)
    f.write(r.content)
print("Download Complete!")

print("Unzipping Enron dataset (This may take a while)")
import tarfile
tfile = tarfile.open("../enron_mail_20150507.tar.gz")
tfile.extractall(".")
tfile.close()

print("You're ready to go!")

Convert all enron email files to csv

In [1]:
import os
import csv

In [2]:
# Define the root directory of the Enron dataset
MAILDIR_PATH = "../maildir" 
OUTPUT_CSV = "enron_emails.csv"

In [3]:
# Define the CSV columns
fields = ["file", "message"]

In [4]:
# Function to extract email content from a file
def extract_email(file_path):
    with open(file_path, "r", encoding="latin1") as file:
        data = file.read()
    return {
        "file": file_path,
        "message": data
    }

# List to store the email data
all_emails = []

# Walk through the directories and extract emails
for root, dirs, files in os.walk(MAILDIR_PATH):
    for file in files:
        file_path = os.path.join(root, file)
        try:
            email_content = extract_email(file_path)
            all_emails.append(email_content)
        except Exception as e:
            print(f"Failed to extract {file_path}: {e}")

# Write the emails to a CSV file
with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as csv_file:
    csv_writer = csv.DictWriter(csv_file, fieldnames=fields)
    csv_writer.writeheader()
    csv_writer.writerows(all_emails)

print(f"Emails have been successfully written to {OUTPUT_CSV}")

Emails have been successfully written to enron_emails.csv


In [20]:
# import needed libraries
import numpy as np
import string
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')
import email
import re
from datetime import datetime
from dateutil import tz
import networkx as nx
import nltk
import wordcloud
import plotly.express as px
from plotly import graph_objects as go

# Machine learning and NLP libraries
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn import metrics
import scipy as sp
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [21]:
# Sentiment Analysis
import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nrclex import NRCLex

Loading data

In [22]:
df = pd.read_csv(r"enron_emails.csv")

In [23]:
df.head()

,file,message
0,/Users/liqi/Desktop/IND5003/5003grp/maildir/.D...,NaN
1,/Users/liqi/Desktop/IND5003/5003grp/maildir/ar...,Message-ID: <17334447.1075857585446.JavaMail.e...
2,/Users/liqi/Desktop/IND5003/5003grp/maildir/ar...,Message-ID: <19171686.1075857585034.JavaMail.e...
3,/Users/liqi/Desktop/IND5003/5003grp/maildir/ar...,Message-ID: <29887033.1075857630725.JavaMail.e...
4,/Users/liqi/Desktop/IND5003/5003grp/maildir/ar...,Message-ID: <29084893.1075849630138.JavaMail.e...


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517403 entries, 0 to 517402
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   file        517403 non-null  object
 1   message     517403 non-null  object
 2   Date        517401 non-null  object
 3   From        517401 non-null  object
 4   To          495554 non-null  object
 5   Subject     517401 non-null  object
 6   X-From      517372 non-null  object
 7   X-To        517372 non-null  object
 8   X-Folder    517372 non-null  object
 9   X-Origin    517372 non-null  object
 10  X-Filename  517372 non-null  object
dtypes: object(11)
memory usage: 43.4+ MB


Data cleaning and preparing

In [28]:
# Write functions to extract mail headers and body
def extractmailitems(field, msg):
    elst = []
    
    for i, message in (msg.items()):
        e = email.message_from_string(message)
        elst.append(e.get(field))
       
    return elst

In [30]:
columns = ['Date', 'From', 'To', 'Subject', 'X-From', 'X-To', 'X-Folder', 'X-Origin', 'X-Filename']
df['message'] = df['message'].astype(str)  # Convert messages to strings
for i in columns:
    df[i] = extractmailitems(i, df['message'])

In [32]:
def body(col):
    bodycolumn = []
    for message in col.values:
        e = email.message_from_string(message)
        bodycolumn.append(e.get_payload())
 
    return bodycolumn
df['Body'] = body(df['message'])

In [33]:
# extract emplyees' names from file column
def employees(files):
    employees = []
    for i, employee in files.items():
        employee = employee.split('/')[0]
        employees.append(employee)
    return employees
df['Employee'] = employees(df['file'])

In [34]:
df.sample(5)

,file,message,Date,From,To,Subject,X-From,X-To,X-Folder,X-Origin,X-Filename,Body,Employee
482313,/Users/liqi/Desktop/IND5003/5003grp/maildir/ke...,Message-ID: <11621426.1075855427794.JavaMail.e...,"Thu, 20 Dec 2001 08:51:28 -0800 (PST)",chris.behney@enron.com,mark.pickering@enron.com,RE: email problem,"Behney, Chris </O=ENRON/OU=NA/CN=RECIPIENTS/CN...","Pickering, Mark </O=ENRON/OU=NA/CN=RECIPIENTS/...","\Steven_Kean_Jan2002\Kean, Steven J.\junk mail",Kean-S,skean (Non-Privileged).pst,Steve is set to unlimited space now. He and hi...,
291686,/Users/liqi/Desktop/IND5003/5003grp/maildir/ma...,Message-ID: <24481601.1075845583729.JavaMail.e...,"Tue, 18 Jul 2000 08:32:00 -0700 (PDT)",aduncan@kilstock.com,jeffrey.m.keenan@enron.com,CPCN and followup,"""Duncan, Allyson"" <aduncan@kilstock.com>","""'Keenan Jeffrey'"" <Jeffrey.M.Keenan@enron.com>",\Kay_Mann_June2001_1\Notes Folders\All documents,MANN-K,kmann.nsf,"Jeffrey, after our discussion yesterday regard...",
35680,/Users/liqi/Desktop/IND5003/5003grp/maildir/ka...,Message-ID: <25824784.1075856373727.JavaMail.e...,"Mon, 27 Nov 2000 03:34:00 -0800 (PST)",vince.kaminski@enron.com,vkaminski@aol.com,Btu Weekly,Vince J Kaminski,vkaminski@aol.com,\Vincent_Kaminski_Jun2001_2\Notes Folders\Disc...,Kaminski-V,vkamins.nsf,---------------------- Forwarded by Vince J Ka...,
488574,/Users/liqi/Desktop/IND5003/5003grp/maildir/ke...,Message-ID: <30286700.1075846190439.JavaMail.e...,"Wed, 8 Nov 2000 01:16:00 -0800 (PST)",steven.kean@enron.com,maureen.mcvicker@enron.com,"Order issued by FERC , EL00-95-000",Steven J Kean,Maureen McVicker,\Steven_Kean_Dec2000_1\Notes Folders\All docum...,KEAN-S,skean.nsf,print\n----- Forwarded by Steven J Kean/NA/Enr...,
121540,/Users/liqi/Desktop/IND5003/5003grp/maildir/ri...,Message-ID: <29790296.1075858647879.JavaMail.e...,"Mon, 22 Oct 2001 21:00:26 -0700 (PDT)",no.address@enron.com,None,New Link for All-Employee Meeting,Public Relations@ENRON,All Enron Worldwide@ENRON,\RRING (Non-Privileged)\Deleted Items,Ring-R,RRING (Non-Privileged)1.pst,Attached is a new link for employees unable to...,


In [35]:
# drop unimportant data columns and empty cells
df.drop(columns = ['file', 'message'], inplace=True)
df.dropna(axis = 0, inplace=True)
df.sample(5)

,Date,From,To,Subject,X-From,X-To,X-Folder,X-Origin,X-Filename,Body,Employee
357946,"Mon, 26 Nov 2001 13:44:15 -0800 (PST)",djcustomclips@djinteractive.com,1529@wctopics.djnr.com,Enron Corp.: CORRECT:Enron Employees' Lawyer B...,djcustomclips@djinteractive.com,1529@WCTOPICS.djnr.com,"\RSHAPIRO (Non-Privileged)\Shapiro, Richard\De...",Shapiro-R,RSHAPIRO (Non-Privileged).pst,"CORRECT:Enron Employees' Lawyer Based In D.C.,...",
36793,"Tue, 22 Feb 2000 18:37:00 -0800 (PST)",vince.kaminski@enron.com,vkaminski@aol.com,"March 2, NYMEX night at the rodeo",Vince J Kaminski,vkaminski@aol.com,\Vincent_Kaminski_Jun2001_7\Notes Folders\Disc...,Kaminski-V,vkamins.nsf,---------------------- Forwarded by Vince J Ka...,
481673,"Tue, 26 Sep 2000 06:32:00 -0700 (PDT)",maureen.mcvicker@enron.com,katherine.brown@enron.com,"Re: EXECUTIVE COMMITTEE MEETING - MONDAY, OCTO...",Maureen McVicker,Katherine Brown,\Steven_Kean_Dec2000_1\Notes Folders\Archiving...,KEAN-S,skean.nsf,STEVE KEAN\n\n\n\n\n\n\tKatherine Brown\n\t09/...,
5315,"Tue, 20 Nov 2001 13:47:13 -0800 (PST)",scott.tholan@enron.com,john.lavorato@enron.com,Resend - Competitive Analysis,"Tholan, Scott </O=ENRON/OU=NA/CN=RECIPIENTS/CN...","Lavorato, John </O=ENRON/OU=NA/CN=RECIPIENTS/C...","\JLAVORA (Non-Privileged)\Lavorato, John\Delet...",Lavorato-J,JLAVORA (Non-Privileged).pst,"John,\nAttached is my prioritized recommendati...",
233697,"Fri, 19 Oct 2001 13:51:20 -0700 (PDT)",mark.whitt@enron.com,"steve.walton@enron.com, paul.kaufman@enron.com...",Buffalo Power Company LLC - Gas Turbine Power ...,"Whitt, Mark </O=ENRON/OU=NA/CN=RECIPIENTS/CN=M...","Walton, Steve </O=ENRON/OU=NA/CN=RECIPIENTS/CN...","\BTYCHOL (Non-Privileged)\Tycholiz, Barry\Dele...",TYCHOLIZ-B,BTYCHOL (Non-Privileged).pst,ENA and Crestone (Northern Border's non-regula...,


In [36]:
df.Employee.value_counts()[:20]

Employee
    495547
Name: count, dtype: int64

In [37]:
# changing the date format and type from string to date object
df['Date'] = pd.to_datetime(df['Date'], utc = True, dayfirst = True)

/var/folders/k8/13s2hdrn4j3c1707gnj300r80000gn/T/ipykernel_45039/2007279768.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'], utc = True, dayfirst = True)


In [ ]:
# creating a year column from the date column
df['Year'] = pd.DatetimeIndex(df['Date']).year

In [ ]:
df.Year.value_counts()

In [ ]:
# a function for cleaning text in columns

def extract(col):
    regcol = []
    for sent in col.values:
        
        if str(sent).startswith('<'):  
            reg = re.split(r'@|\(', str(sent))[0]
            reg = re.findall(r'[a-zA-Z]+\'?-?', str(reg))           
            
        elif re.match(r'^\d+', str(sent)):
            reg = re.split(r'@|\(', str(sent))[0]
            reg = re.findall(r'[0-9]+\'?-?', str(reg))
           
        else:
            reg = re.split(r'@|<|\(', str(sent))[0]
            reg = re.findall(r'[a-zA-Z]+\'?-?', str(reg))
            
        reg = re.sub(r'[\'\",]', '', str(reg))
        regcol.append(str(reg.strip('[]')))
           
            
    return regcol

In [ ]:
df['X-From'] = extract(df['X-From'])
df['X-To'] = extract(df['X-To'])

In [ ]:
df['X-From'].value_counts()[:20]

In [ ]:
df['X-To'].value_counts()[:20]

In [ ]:
df.sample(5)

In [ ]:
df.info()

In [ ]:
# make a diagram to show the count of messages sent per year
ax = df.groupby(df.Year)['X-Origin'].count().plot(figsize = (8,6))
ax.set_xlim(1995,2005)
ax.set_xlabel('Year', fontsize = 16)
ax.set_ylabel('Count', fontsize = 16)
ax.set_title('Messages sent across the years', fontsize = 16)
plt.show()

In [ ]:
# diagram for showing top senders
plt.figure(figsize = (16,10))
ax = sns.barplot(x = df['X-Origin'].value_counts().values[:30], y = df['X-Origin'].value_counts().keys()[:30] ,palette = 'rocket', orient = 'h')
ax.set_xlabel('Count', fontsize = 18)
ax.set_ylabel('Original Senders', fontsize = 18)
ax.set_title('Top 30 Senders', fontsize = 20)
plt.show()

In [ ]:
# diagram showing top recepients
plt.figure(figsize = (16,10))
ax = sns.barplot(x = df['To'].value_counts().values[:30], y = df['To'].value_counts().keys()[:30] ,palette = 'crest', orient = 'h')
ax.set_xlabel('Count', fontsize = 18)
ax.set_ylabel('Recepient E-Mail', fontsize = 18)
ax.set_title('Top 30 Recepient E-Mails', fontsize = 20)
plt.show()

In [ ]:
userlist = df['X-Origin'].unique()

In [ ]:
useryears = []
for user in userlist:
    year = df.loc[df['X-Origin'] == user, 'Year'].iloc[0]
    useryears.append(year)
    useryears

In [ ]:
usercount = []
counter = 0
for name in userlist:
    for user in df['X-Origin'].values:
        if user == name:
            counter += 1
    usercount.append(counter)
    counter = 0

In [ ]:
dataf = sorted(list(zip(userlist, usercount, useryears)), key= lambda user: user[1], reverse = True)[:30]

In [ ]:
dataf = pd.DataFrame(dataf)
dataf.columns = ['User', 'Count', 'Year']
dataf.head()

In [ ]:
# diagram showing top users and count of mails sent per year
plt.figure(figsize = (20,16))
ax = sns.barplot(data = dataf, x ='User', y = 'Count', hue = 'Year', palette = 'viridis' , saturation = 0.7, width = 1.5)
sns.despine()
plt.xlabel('User', fontsize = 18)
plt.ylabel('Count', fontsize = 18)
plt.title('User sent mails/year', fontsize = 22)
plt.xticks(rotation = 45, fontsize = 12)
plt.yticks(fontsize = 12)
plt.legend(fontsize = 14)
plt.show()

In [ ]:
# a network of first 2500 rows showing the network of Allen P
plt.figure(figsize = (20, 20))
G = nx.from_pandas_edgelist(df[:2500], 'X-Origin', 'X-To')
pos = nx.draw_random(G, node_size = 50, node_color = 'blue', edge_color = 'salmon', with_labels = True)
plt.title('Network of Emails (First 2500)', fontsize = 24)
plt.show()